In [ ]:

# model_id = 'pierreguillou/t5-base-qa-squad-v1.1-portuguese'
# model_id = 'unicamp-dl/ptt5-small-t5-vocab'
# model_id = 'unicamp-dl/ptt5-base-portuguese-vocab'

In [ ]:

from transformers import T5Tokenizer, T5ForConditionalGeneration

import torch
from pipelines import pipeline
device = 'cuda' if torch.cuda.is_available() else 'cpu'


model_id = 'vabatista/question-generation-t5-base-pt-br'
tokenizer = T5Tokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id).to(device)


nlp = pipeline("question-generation", model=model, tokenizer=tokenizer, ans_model=model, ans_tokenizer=tokenizer)

text = """A Volkswagen anunciou a chegada do ID.Buzz, a Kombi elétrica, ao Brasil. Em campanha publicitária, a marca alemã usou tecnologia de inteligência artificial 
para criar um comercial com a cantora Elis Regina, falecida em 1982, e a sua filha, a também cantora Maria Rita. Ambas aparecem cantando juntas a música 'Como Nossos Pais', composta por Belchior e eternizada por Elis.
O vídeo, que já foi divulgado nas redes sociais da marca, foi exibido pela primeira vez em comemoração de 70 anos da Volkswagen no ginásio do Ibirapuera, em São Paulo.
Diante de 5 mil pessoas, entre funcionários e convidados, a apresentação ainda contou com a presença de Maria Rita, que também cantou ao vivo a canção e se emocionou bastante - 
a cantora chegou a chorar abraçada com Ciro Possobom, CEO da VW do Brasil.
A técnica utilizada, conhecida também como "deep fake", aplica IA para criar conteúdos realistas. No caso, foi produzida pela agência AlmapBBDO."""

nlp(text)


/home/r/anaconda3/envs/study/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


[{'answer': 'ID.Buzz',
  'question': 'Qual é o nome da Kombi elétrica da Volkswagen no Brasil?'},
 {'answer': 'Elis Regina',
  'question': 'Com quem a Volkswagen fez um comercial com ID.Buzz?'},
 {'answer': 'Belchior',
  'question': "Quem compôs a música 'Como Nossos Pais'?"},
 {'answer': 'ginásio do Ibirapuera',
  'question': 'Onde foi exibido o comercial ID.Buzz pela primeira vez no Brasil?'},
 {'answer': 'Ciro Possobom', 'question': 'Quem foi o CEO da VW no Brasil?'},
 {'answer': 'deep fake',
  'question': 'Qual é o outro nome da técnica usada para criar conteúdo realista?'},
 {'answer': 'AlmapBBDO',
  'question': "Quem produziu o comercial da Volkswagen com a música 'Como Nossos Pais'?"}]

In [ ]:
from time import time
from nltk import sent_tokenize

text = """A Volkswagen anunciou a chegada do ID.Buzz, a Kombi elétrica, ao Brasil. Em campanha publicitária, a marca alemã usou tecnologia de inteligência artificial 
para criar um comercial com a cantora Elis Regina, falecida em 1982, e a sua filha, a também cantora Maria Rita. Ambas aparecem cantando juntas a música 'Como Nossos Pais', composta por Belchior e eternizada por Elis.
O vídeo, que já foi divulgado nas redes sociais da marca, foi exibido pela primeira vez em comemoração de 70 anos da Volkswagen no ginásio do Ibirapuera, em São Paulo.
Diante de 5 mil pessoas, entre funcionários e convidados, a apresentação ainda contou com a presença de Maria Rita, que também cantou ao vivo a canção e se emocionou bastante - 
a cantora chegou a chorar abraçada com Ciro Possobom, CEO da VW do Brasil.
A técnica utilizada, conhecida também como "deep fake", aplica IA para criar conteúdos realistas. No caso, foi produzida pela agência AlmapBBDO."""

sentences = sent_tokenize(text)
print('start')
s = time()
for sentence in sentences:
    extract_answer = [f'extrair respostas: {sentence}']
    inputs = tokenizer(sentence, return_tensors='pt').to(device)
    # # Generate Summary
    summary_ids = model.generate(inputs["input_ids"])

    answer = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
    # print(answer)
    # t = text.replace(answer, f'<hl> {answer} <hl>')
    t = sentence.replace(answer, f'<hl> {answer} <hl>')
    # print(t)
    inputs = tokenizer(f'gerar perguntas: {t}', return_tensors='pt').to(device)
    # print(sentence.replace(answer, f' <hl> {answer} <hl> '))
    summary_ids = model.generate(inputs['input_ids'])
    question = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)

    print(f'pergunta: {question}\nresposta: {answer}\n')
print(f'end, elapsed time: {time()-s:.4f} seconds')

start


/home/r/anaconda3/envs/study/lib/python3.10/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


pergunta: Qual é o nome da Kombi elétrica que a Volkswagen anunciou no Brasil?
resposta: ID.Buzz

pergunta: Com quem a marca alemã usou tecnologia de inteligência artificial para criar um comercial?
resposta: Elis Regina, falecida em 1982, e a sua filha, a também cantora Maria Rita.

pergunta: Quem compôs a música 'Como Nossos Pais'?
resposta: Belchior e eternizada por Elis.

pergunta: Onde foi exibido o vídeo?
resposta: o ginásio do Ibirapuera, em São Paulo.

pergunta: Maria Rita cantou a música e se emocionou bastante com Ciro Possobom, CEO
resposta: Maria Rita, que também cantou ao vivo a canção e se emocionou bastante - a cantora

pergunta: O que o deep fake aplica?
resposta: IA para criar conteúdos realistas

pergunta: Qual agência produziu o caso?
resposta: AlmapBBDO

end, elapsed time: 4.0513 seconds


In [ ]:
# pergunta: Qual é o nome da Kombi elétrica que a Volkswagen anunciou no Brasil?
# resposta: ID.Buzz

# pergunta: Com quem a marca alemã usou tecnologia de inteligência artificial para criar um comercial?
# resposta: Elis Regina, falecida em 1982, e a sua filha, a também cantora Maria Rita.

# pergunta: Quem compôs a música 'Como Nossos Pais'?
# resposta: Belchior e eternizada por Elis.

# pergunta: Onde foi exibido o vídeo?
# resposta: o ginásio do Ibirapuera, em São Paulo.

# pergunta: Maria Rita cantou a música e se emocionou bastante com Ciro Possobom, CEO
# resposta: Maria Rita, que também cantou ao vivo a canção e se emocionou bastante - a cantora

# pergunta: O que o deep fake aplica?
# resposta: IA para criar conteúdos realistas

# pergunta: Qual agência produziu o caso?
# resposta: AlmapBBDO

# end 4.377967119216919